In [1]:
import sys
if '/usr/users/vogg/.conda/envs/mktrack/lib/python3.8/site-packages' not in sys.path:
    sys.path.insert(0, '/usr/users/vogg/.conda/envs/mktrack/lib/python3.8/site-packages')
    sys.path.insert(0, '/usr/users/vogg/monkey-tracking-in-the-wild/src')
    sys.path.insert(0, '/usr/users/vogg/monkey-tracking-in-the-wild/src/lib')

import cv2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
import torchvision.transforms as T
import torchvision
from torchmetrics.detection import MeanAveragePrecision

import datasets.dataset as datasets
from models.model import create_model, load_model
from models.decode import mot_decode, map2orig

from opts import opts

In [2]:
models = ['macaquepose', 'macaquecp', 'macaquecpw']
seeds = ['1','2','3']
valsets = ['MacaquePose', 'MacaqueCopyPaste', 'MacaqueCopyPasteWild']

In [3]:
df = pd.DataFrame(columns = ['model', 'valset', 'seed', 'AP', 'AP50', 'AP75'])


for mdl in models:
    for valset in valsets:
        for seed in seeds:
            epoch = 200
            print(mdl, valset, seed)

            model = create_model('hrnet_32', heads =  {'hm': 1, 'wh': 2, 'id': 128, 'reg': 2}, 
                                 head_conv = 256,num_gc_cls = None, 
                                 clsID4GC = None, gc_with_roi = False)
            
            if mdl == 'macaquepose':
                epoch *= 2

            #model = load_model(model, '../models/mcqcp/model_150.pth')
            model = load_model(model, '../exp/mot/'+mdl+'_seed'+seed+'/model_'+str(epoch)+'.pth')
            model = model.to(torch.device('cuda'))
            model.eval()

            path = "/usr/users/agecker/datasets/"

            dataloader = datasets.LoadImagesAndBoxes(path, 
                                                     "/usr/users/vogg/monkey-tracking-in-the-wild/src/data/macaquepose.val",
                                                    valset = valset)

            mean_ap = MeanAveragePrecision()

            for i, (img_path, img, img0,labels) in enumerate(dataloader):
                if i == 500:
                    break


                im_blob = torch.from_numpy(img).cuda().unsqueeze(0)

                width = img0.shape[1]
                height = img0.shape[0]
                inp_height = im_blob.shape[2]
                inp_width = im_blob.shape[3]
                h_out = inp_height // 4
                w_out = inp_width // 4


                with torch.no_grad():
                    output = model(im_blob)[-1]
                    hm = output['hm'].sigmoid_()
                    wh = output['wh']
                    id_feature = output['id']
                    id_feature = F.normalize(id_feature, dim=1)
                    reg = output['reg']

                dets, inds, cls_inds_mask = mot_decode(heatmap=hm,
                                                       wh=wh,
                                                       reg=reg,
                                                       num_classes=1,
                                                       cat_spec_wh=False,
                                                       K=50)

                # translate and scale
                dets = map2orig(dets, h_out, w_out, height, width, 1)
                dets = dets[0]

                remain_inds = dets[:, 4] > 0.01
                dets = dets[remain_inds]
                #print("Total detections: ", len(dets), " - gt: ", len(labels))

                #ground truth
                boxes = torch.Tensor(labels[:,2:6] * np.array([width, height, width, height]))
                boxes = torchvision.ops.box_convert(boxes, "cxcywh", "xyxy")



                mean_ap.update(
                    [dict(boxes=torch.Tensor(dets[:,0:4]), scores=torch.Tensor(dets[:,4]), labels=torch.ones(len(dets[:,5])))],
                    [dict(boxes=boxes, labels=torch.ones(len(boxes)))],
                )

            scores = mean_ap.compute()
            print({'model': mdl, 'valset': valset, 'seed': seed, 
                      'AP': scores['map'], 'AP50': scores['map_50'], 'AP75': scores['map_75']})
            df = pd.concat([df, pd.DataFrame({'model': mdl, 'valset': valset, 'seed': seed, 
                      'AP': scores['map'].numpy(), 'AP50': scores['map_50'].numpy(), 'AP75': scores['map_75'].numpy()}, index = [0])])

                #img0 = cv2.cvtColor(img0, cv2.COLOR_BGR2RGB)
                #plt.imshow(img0)
                #plt.show()

macaquepose MacaquePose 1
{'hm': 1, 'wh': 2, 'id': 128, 'reg': 2}
loaded ../exp/mot/macaquepose_seed1/model_400.pth, epoch 400
{'model': 'macaquepose', 'valset': 'MacaquePose', 'seed': '1', 'AP': tensor(0.7084), 'AP50': tensor(0.9207), 'AP75': tensor(0.8375)}
macaquepose MacaquePose 2
{'hm': 1, 'wh': 2, 'id': 128, 'reg': 2}
loaded ../exp/mot/macaquepose_seed2/model_400.pth, epoch 400
{'model': 'macaquepose', 'valset': 'MacaquePose', 'seed': '2', 'AP': tensor(0.7047), 'AP50': tensor(0.9138), 'AP75': tensor(0.8141)}
macaquepose MacaquePose 3
{'hm': 1, 'wh': 2, 'id': 128, 'reg': 2}
loaded ../exp/mot/macaquepose_seed3/model_400.pth, epoch 400
{'model': 'macaquepose', 'valset': 'MacaquePose', 'seed': '3', 'AP': tensor(0.7191), 'AP50': tensor(0.9250), 'AP75': tensor(0.8383)}
macaquepose MacaqueCopyPaste 1
{'hm': 1, 'wh': 2, 'id': 128, 'reg': 2}
loaded ../exp/mot/macaquepose_seed1/model_400.pth, epoch 400
{'model': 'macaquepose', 'valset': 'MacaqueCopyPaste', 'seed': '1', 'AP': tensor(0.6228)

In [4]:
df['AP'] = [x.numpy() for x in df['AP']]
df['AP50'] = [x.numpy() for x in df['AP50']]
df['AP75'] = [x.numpy() for x in df['AP75']]

AttributeError: 'float' object has no attribute 'numpy'

In [5]:
df

,model,valset,seed,AP,AP50,AP75
0,macaquepose,MacaquePose,1,0.708365,0.920696,0.837462
0,macaquepose,MacaquePose,2,0.704698,0.913848,0.814069
0,macaquepose,MacaquePose,3,0.719134,0.924964,0.838260
0,macaquepose,MacaqueCopyPaste,1,0.622787,0.858684,0.709374
0,macaquepose,MacaqueCopyPaste,2,0.614179,0.844319,0.709203
0,macaquepose,MacaqueCopyPaste,3,0.622862,0.846804,0.714167
0,macaquepose,MacaqueCopyPasteWild,1,0.710832,0.914292,0.823229
0,macaquepose,MacaqueCopyPasteWild,2,0.719135,0.918790,0.822644
0,macaquepose,MacaqueCopyPasteWild,3,0.732940,0.919182,0.838149
0,macaquecp,MacaquePose,1,0.731173,0.926342,0.848983


In [6]:
df.to_csv("/usr/users/vogg/monkey-tracking-in-the-wild/models200_detection.csv")

In [39]:
len(boxes)

2

In [41]:
torch.Tensor(dets[:,0:4])

tensor([[  20.4627,  272.1553,  458.3413,  672.3766],
        [ 588.6373,   68.4057, 1009.8489,  637.6304]])

{'map': tensor(0.6525),
 'map_50': tensor(1.),
 'map_75': tensor(0.5050),
 'map_small': tensor(-1.),
 'map_medium': tensor(-1.),
 'map_large': tensor(0.6525),
 'mar_1': tensor(0.4500),
 'mar_10': tensor(0.6500),
 'mar_100': tensor(0.6500),
 'mar_small': tensor(-1.),
 'mar_medium': tensor(-1.),
 'mar_large': tensor(0.6500),
 'map_per_class': tensor(-1.),
 'mar_100_per_class': tensor(-1.)}